# Connect Player `starting` in gams

In [82]:
import os
os.chdir("/home/jovyan/work")

from neomodel import (
    BooleanProperty,
    DateTimeProperty,
    FloatProperty,
    IntegerProperty,
    RelationshipTo, 
    StringProperty, 
    StructuredNode, 
    StructuredRel,
)
import pandas as pd
from tqdm import tqdm

from src.models import Player, Game
from src.utils import init_connection

In [54]:
class StartsInRel(StructuredRel):
    for_team = StringProperty()

In [55]:
class Player(StructuredNode):
    """Boilerplate for a player node
    
    Attributes:
        msfID (int): My-Sports-Feed ID of the player
        firstName (str): first name of the player
        lastName (str): last name of the player
        primaryPosition (str): primary position 
        height (float): height of the player in cm
        weight (float): weight of the player in kg
        birthDate (datetime): day of birth
        rookie (bool): flag indicating if the player is a rookie
        shoots (str): primary throwing hand
        officialImageSrc (str): URL of the official NBA "mugshot"
        draftedYear (int): year the player was drafted
        draftedRound (int): round the player was drafted
        draftedRoundPick (int): position the player was drafted inside the draft round
        draftedOverallPick (int): position the player was drafted overall
        nbaID = (int): stats.nba ID of the player
    """

    # PROPERTIES
    msfID = IntegerProperty(required=True, unique_index=True)
    firstName = StringProperty(required=True)
    lastName = StringProperty(required=True)
    primaryPosition = StringProperty(required=True)
    height = FloatProperty(required=True)
    weight = FloatProperty(required=True)
    birthDate = DateTimeProperty(required=True)
    rookie = BooleanProperty(require=True)
    shoots = StringProperty()
    officialImageSrc = StringProperty(required=True)
    draftedYear = IntegerProperty(required=True)
    draftedRound = IntegerProperty(required=True)
    draftedRoundPick = IntegerProperty(required=True)
    draftedOverallPick = IntegerProperty(required=True)
    nbaID = IntegerProperty()
    
    # RELATIONSHIP
    startsIn = RelationshipTo("Game", "STARTS_IN", model=StartsInRel)

In [56]:
df_lu = pd.read_csv("./data/lineups.csv")

df_lu.head()

,game_id,p_01,p_02,p_03,p_04,p_05,p_06,p_07,p_08,p_09,p_10,t_1,t_2
0,31176,9207,9400,9213,9202,9203,9086,9093,9091,9082,9089,DET,ATL
1,31177,9158,9161,9163,9160,9167,9145,9146,9142,9149,9152,CLE,CHI
2,31178,9353,9357,9364,9363,9368,9219,9230,9220,9218,9228,NOP,GSW
3,31188,9524,9096,9522,9519,9523,9399,9212,9407,9412,9402,WAS,ORL
4,31179,9416,9413,9414,9421,9200,9107,9186,9102,9099,9097,PHI,BOS


In [57]:
init_connection()

In [80]:
["p_{:02d}".format(n) for n in list(range(6,11))]

['p_06', 'p_07', 'p_08', 'p_09', 'p_10']

In [87]:
for _, row in tqdm(df_lu.iterrows(), total=df_lu.shape[0]):
    g = Game.nodes.get(game_id=row["game_id"])
    for p1 in ["p_{:02d}".format(n) for n in list(range(1,6))]:
        Player.nodes.get(msfID=row[p1]).startsIn.connect(g, {"for_team":row["t_1"]})
    for p2 in ["p_{:02d}".format(n) for n in list(range(6,11))]:
        Player.nodes.get(msfID=row[p2]).startsIn.connect(g, {"for_team":row["t_2"]})

100%|██████████| 3855/3855 [05:14<00:00, 12.24it/s]
